In [2]:
!pip install pendulum

In [54]:
import modules.s3_module as s3_module
import re
import pyarrow.fs as fs
from modules.config import Musinsa_Config
import pyarrow.parquet as pq

s3 = s3_module.connect_s3fs()

def connect_s3fs():
    s3 = fs.S3FileSystem(
        access_key=AWS_Config.AWS_ACCESS_KEY_ID,
        secret_key=AWS_Config.AWS_SECRET_ACCESS_KEY,
        region=AWS_Config.REGION,
        endpoint_override='https://s3.ap-northeast-2.amazonaws.com'  
    )
    return s3

def get_product_ids(s3_path):
    dataset = pq.ParquetDataset(s3_path, filesystem=connect_s3fs())
    table = dataset.read(columns=['product_id'])
    
    product_ids = table['product_id'].to_pylist()
    
    return product_ids

bucket_path = "project4-silver-data/"

file_key = f"{Musinsa_Config.today_date}/Musinsa/RankingData/전체/"

path = bucket_path + file_key

files = s3.get_file_info(fs.FileSelector(base_dir=path, recursive=True))

category4depth = "스니커즈"
# 패턴에 맞는 파일 경로 리스트 생성

directory_pattern = re.compile(rf'.*{category4depth}\.parquet$')
directories = [file for file in files if file.type == fs.FileType.Directory and directory_pattern.match(file.path)]

product_ids = []
for directory in directories:
    temp_ids = get_product_ids(directory.path)
    product_ids += temp_ids
    
len(set(product_ids)

79

In [ ]:
import threading
import time
import requests

test_list = [3576758, 4002983, 4500095, 3583663, 4209447, 4278902, 4516351, 4553898, 4506476, 4280287, 3238559, 3450857, 3771691, 3518760, 3812667, 4344259, 4440000, 4299540, 4375133, 3836735, 4320212, 1223884, 4287892, 4403558, 4420951, 4252204, 4308610, 4287419, 4287413, 4287410, 1790382, 3622977, 401149, 3466541, 3802459, 4300024, 3059521, 3898494, 3899228, 4252198, 3466533, 3046279, 4300012, 3832994, 4344269, 4300049, 4386389, 4009288, 3065000, 2775794]

def chunked_iterable(iterable, size):
    """리스트를 size 크기만큼 쪼개서 반환하는 제너레이터 함수"""
    for i in range(0, len(iterable), size):
        yield iterable[i:i + size]

def do_something(id_list):
    HEADERS = {
        "accept": "application/json, text/plain, */*",
        "accept-encoding": "gzip, deflate, br, zstd",
        "accept-language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
        "origin": "https://www.musinsa.com",
        "referer": "https://www.musinsa.com/",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
        "sec-ch-ua": '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": '"Windows"'
    }
    
    params = {
        "page": 0,
        "pageSize": 20,
        "sort": "new",
        "myFilter": "false",
        "hasPhoto": "false",
        "isExperience": "false"
    }
    
    url = "https://goods.musinsa.com/api2/review/v1/view/list"

    for product_id in id_list:
        params["goodsNo"] = product_id
        response = requests.get(url, headers=HEADERS, params=params)
        print(response.json())
        time.sleep(0.5)

if __name__ == '__main__':
    start = time.perf_counter()
    
    # 스레드를 담을 리스트 threads 초기화
    threads = []
    
    # test_list를 10개씩 쪼개서 각 청크에 대해 스레드를 생성하고 실행
    for chunk in chunked_iterable(test_list, 10):
        t = threading.Thread(target=do_something, args=(chunk,))
        t.start()
        threads.append(t)
    
    # 모든 스레드가 실행이 완료된 후 다음 코드를 실행시키기 위하여 join을 이용
    for thread in threads:
        thread.join()
    
    finish = time.perf_counter()
    
    print(f'Finished in {round(finish-start, 2)} second(s)')

{'data': {'list': [{'no': 66281259, 'type': 'style', 'typeName': '스타일', 'content': '옷이 두꺼워서 초겨울까지는 갠찮을고같어요', 'commentCount': 0, 'grade': '0', 'goods': {'goodsNo': 3576758, 'goodsSubNo': 0, 'goodsDiscountRate': 0.0, 'goodsName': '데님 블루종 점퍼 블랙', 'goodsImageFile': '/images/goods_img/20230920/3576758/3576758_16975894062482_100.jpg', 'goodsImageExtension': 'jpg', 'goodsPrice': 179000, 'goodsNormalPrice': 179000, 'goodsOptionKindCode': 'CLOTHES', 'brandName': '리', 'brandEnglishName': 'LEE', 'brand': 'lee', 'brandBestYn': 'Y', 'brandConcatenation': 'lee@#리@#LEE', 'goodsCreateDate': '2023-09-20T12:43:03.000+09:00', 'goodsImageIdx': 0, 'saleStatCode': None, 'saleStatLabel': None, 'goodsSex': 0, 'goodsSexClassification': None, 'showSoldOut': False}, 'userImageFile': '/mfile_s01/_simbols/_basic/basic.png', 'goodsOption': 'XL', 'commentReplyCount': 0, 'userStaffYn': 'N', 'images': [{'altText': '리(LEE) 데님 블루종 점퍼 블랙 후기', 'imageUrl': '/data/estimate/3576758_0/ff8a694d2957415784de0a8eb98e9db1.jpg'}], 

In [57]:
do_something(test_list)

{'data': {'list': [{'no': 66281259, 'type': 'style', 'typeName': '스타일', 'content': '옷이 두꺼워서 초겨울까지는 갠찮을고같어요', 'commentCount': 0, 'grade': '0', 'goods': {'goodsNo': 3576758, 'goodsSubNo': 0, 'goodsDiscountRate': 0.0, 'goodsName': '데님 블루종 점퍼 블랙', 'goodsImageFile': '/images/goods_img/20230920/3576758/3576758_16975894062482_100.jpg', 'goodsImageExtension': 'jpg', 'goodsPrice': 179000, 'goodsNormalPrice': 179000, 'goodsOptionKindCode': 'CLOTHES', 'brandName': '리', 'brandEnglishName': 'LEE', 'brand': 'lee', 'brandBestYn': 'Y', 'brandConcatenation': 'lee@#리@#LEE', 'goodsCreateDate': '2023-09-20T12:43:03.000+09:00', 'goodsImageIdx': 0, 'saleStatCode': None, 'saleStatLabel': None, 'goodsSex': 0, 'goodsSexClassification': None, 'showSoldOut': False}, 'userImageFile': '/mfile_s01/_simbols/_basic/basic.png', 'goodsOption': 'XL', 'commentReplyCount': 0, 'userStaffYn': 'N', 'images': [{'altText': '리(LEE) 데님 블루종 점퍼 블랙 후기', 'imageUrl': '/data/estimate/3576758_0/ff8a694d2957415784de0a8eb98e9db1.jpg'}], 